In [6]:
#pip install polygon-api-client

Note: you may need to restart the kernel to use updated packages.


In [14]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [108]:
#pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [1]:
#pip install yahooquery
#pip install beautifulsoup4 requests
#pip install yahoo_fin
#pip install stocksymbol

In [4]:
pip install eodhd -U

Note: you may need to restart the kernel to use updated packages.


In [55]:
# test run for finnhub
import requests
import json

#ESTABLISH API KEY
API_KEY = "chdl0fpr01qk9rb2lck0chdl0fpr01qk9rb2lckg"

#Specify stock symbol 
symbol = "AAPL"

#Build the API request URL
url = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={API_KEY}'

response = requests.get(url)
data = json.loads(response.text)
print(data)

{'c': 175.16, 'd': 0.11, 'dp': 0.0628, 'h': 176.29, 'l': 174.95, 'o': 176.39, 'pc': 175.05, 't': 1684526404}


In [19]:
import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np
from datetime import datetime
import time
import traceback

#All relevant methods for API calls

#Checking its existence (see whether can try to beautify)
def check_existence(stock):
    skip = True
    output = Ticker(stock).price[stock]
    if "Quote not found for ticker symbol:" in output:
        print(stock, "is not found/recognised, SKIPPING")
        skip = False
    return skip

#Getting relevant data for each of the columns/ segments
def get_price_marketCap(stock):
    tick = Ticker(stock)
    ticker = tick.price[stock]
    price = ticker['regularMarketPrice']
    marketCap = ticker['marketCap']
    return price, marketCap

#Testing get_price_marketCap
#print(get_price_marketCap('aapl'))

def get_outstandingShares_enterpriseValue_peg(stock):
    tick = Ticker(stock)
    ticker = tick.key_stats[stock]
    shares_outstanding = ticker['sharesOutstanding']
    enterprise_val = ticker['enterpriseValue']
    
    #If peg does not have a value
    try:
        peg = ticker['pegRatio']
    except:
        print('Invalid PEG Ratio for', stock)
        peg = None
        
    return shares_outstanding, enterprise_val, peg

#Testing get_outstandingShares_enterpriseValue_peg
#print(get_outstandingShares_enterpriseValue_peg('aapl'))

def get_totalDebt_totalCash_EBITDA(stock):
    tick = Ticker(stock)
    ticker = tick.financial_data[stock]
    try:
        debt = ticker['totalDebt']
    except:
        debt = 0
        
    try:
        cash = ticker['totalCash']
    except:
        cash = 0
        
    try:
        ebitda = ticker['ebitda']
    except:
        ebitda = 0
        
    return debt, cash, ebitda

#Testing get_totalDebt_totalCash_EBITDA
#print(get_totalDebt_totalCash_EBITDA('aapl'))

def get_dilutedEps_revenue(stock):
    tick = Ticker(stock)
    data = tick.all_financial_data()
    index_last = len(data) - 1
    diluted_eps = data.iloc[index_last]['DilutedEPS']
    revenue = data.iloc[index_last]['TotalRevenue']
    return diluted_eps, revenue

#Testing get_dilutedEps_revenue
#print(get_dilutedEps_revenue('aapl'))

def get_quarterlyRevenueGrowth(stock):
    tick = Ticker(stock)
    print(stock)
    data = tick.earnings[stock]['financialsChart']['quarterly']
    total_data_num = len(data)
    starting_count = 0
    if data[len(data) - 1]['revenue'] == 0:
        starting_count = - 1
    if total_data_num >= 2:
        latest_quarter = data[len(data) - 1 + starting_count]['revenue']
        quarter_before = data[len(data) - 2 + starting_count]['revenue']
        quarterly_revenue_growth = round((latest_quarter - quarter_before) / quarter_before * 100, 2)
    else:
        quarterly_revenue_growth = 'No Data'
    return quarterly_revenue_growth


def express_in_MM(number):
    return number/1_000_000

#Can shift column headers accordingly
def column_headers():
    return ['COMPANY NAME', 'SHARE PRICE ($/share)', 'OUTSTANDING SHARES', 
                      'MARKET CAP ($M)', 'TOTAL DEBT ($M)', 'TOTAL CASH ($M)',
                     'DILUTED EPS ($/share)', 'ENTERPRISE VALUE ($)', 'REVENUE ($)',
                     'QUARTERLY REVENUE GROWTH (%)', 'EBITDA ($M)', 'EBITDA MARGIN (%)',
                     'EV/REVENUE (x)', 'EV/EBITDA (x)', 'PEG 5Y Expected (x)']
def create_df(): 
    return pd.DataFrame(columns = column_headers())

def change_to_dictionary(data_list):
    dic = {}
    #list_columns is from main >> when creating the data frame
    # Can choose to create one here or one in the main frame
    
    for i, col_name in enumerate(column_headers()):
        dic[col_name] = data_list[i]
    return dic

def check_EBITDA(stock):
    status = True
    try:
        Ticker(stock).financial_data[stock]['ebitda']
    except:
        status = False
    return status

def get_all_data(stock):
    company_name = stock
    share_price, market_cap = get_price_marketCap(stock)
    outstanding_shares, enterprise_v, peg = get_outstandingShares_enterpriseValue_peg(stock)
    total_debt, total_cash, ebitda = get_totalDebt_totalCash_EBITDA(stock)
    ev = total_debt + market_cap
    diluted_eps, revenue = get_dilutedEps_revenue(stock)
    quarterly_revenue_growth = get_quarterlyRevenueGrowth(stock)
    ebitda_margin = round(ebitda/revenue * 100, 2)
    ev_revenue = round(ev / revenue, 2)
    if ebitda == 0:
        ev_ebitda = 0
    else:
        ev_ebitda = round(ev / ebitda, 2)
    ordered_list = [company_name, share_price, outstanding_shares, round(express_in_MM(market_cap), 2),
                   round(express_in_MM(total_debt), 2), round(express_in_MM(total_cash), 2), diluted_eps, ev,
                   revenue, quarterly_revenue_growth, round(express_in_MM(ebitda), 2), 
                   ebitda_margin, ev_revenue, ev_ebitda, peg]
    return ordered_list

#print(get_quarterlyRevenueGrowth('aapl'))


In [28]:
#Scrape as last resort
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json
import urllib.request
import ssl


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
    response = urllib.request.urlopen(url, context=ssl_context)
    data = response.read().decode("utf-8")
    return json.loads(data)

#Identification of the current stock's industry
api_key = 'd9d5e433dd4f6a8dabb311f2c98941d0'

def scrape_peer_universe(stock):
    url = f"https://financialmodelingprep.com/api/v3/profile/{stock}?apikey={api_key}"
    industry = get_jsonparsed_data(url)[0]['industry'].split(" ")[0]
    sector = get_jsonparsed_data(url)[0]['sector'].split(" ")[0]

    #Identification of Stocks in Industry
    url_industry = f'https://financialmodelingprep.com/api/v3/stock-screener?sector={sector}&industry={industry}&limit=100&apikey={api_key}'
    exchange_lst = ['NYSE', 'NASDAQ']
    industry1 = list(filter(lambda x: x['exchangeShortName'] in exchange_lst, get_jsonparsed_data(url_industry)))

    #Can explore to include more parameters here
    #Selection of top 5 market cap firms to serve as the peer universe
    industry_sorted = sorted(industry1, key = lambda x : x['marketCap'], reverse = True)[:5]
    
    #May have to limit to NYSE and NASDAQ >> as model may not be able to look into european markets
    peer_universe = list(map(lambda x: x['symbol'], industry_sorted))
    return peer_universe


In [29]:
#Identifying correct peer universe
import yfinance as yf 
from yahooquery import Screener

def not_correct_industry(stock, industry):
    return not yf.Ticker(stock).info['industry'] == industry

#Peer Universe provided by YahooQuery's recommendation function
def og_peer_universe(stock, industry):
    p_universe = []
    tickers = Ticker(stock).recommendations[stock]['recommendedSymbols']
    
    for dic in tickers:
        if not_correct_industry(dic['symbol'], industry):
            continue
        p_universe.append(dic['symbol'])
    
    return p_universe

#Recommendations provided by YahooQuery is applicable in the CCA valuations
def lucky_peer_universe(lst, industry):
    if lst == []:
        return []
    elif og_peer_universe(lst[0], industry) == []:
        print(type(lst))
        return [lst[0], ] + peer_universe(lst[1:], industry)
    else:
        tickers = og_peer_universe(lst[0], industry)
        return tickers + lucky_peer_universe(lst[1:], industry)

#Recommendations in the event YahooQuery's Recommendations function is not comprehensive enough
def peer_universe_(stock, industry):
    lst = og_peer_universe(stock, industry)
    if lucky_peer_universe(lst, industry) != []:
        #explore parameters to include here >> but just gonna go with the top 5
        return list(set(lucky_peer_universe(lst, industry)))[:5]
    else:
        return scrape_peer_universe(stock.upper())


In [30]:
#Main >> To execute main code
df = create_df()

#Generation of Peer Universe
stock = input('Input Company Ticker Symbol: ')
stock = stock.lower()
stock_industry = yf.Ticker(stock).info['industry']

#Check Validity of Ticker
if check_existence(stock):
    # develop peer universe via the yahooquery API
    peer_universe = peer_universe_(stock, stock_industry)
    print(peer_universe)
    if stock.upper() in peer_universe:
        peer_universe.remove(stock.upper())
    peer_universe.append(stock.upper())

    #Loop through peer universe list + create dataframe
    for ticker in peer_universe:
        print("Getting data for: ", ticker)
        if not check_EBITDA(ticker):
            continue
        ticker_data_dict = change_to_dictionary(get_all_data(ticker))
        df_result_temp = pd.DataFrame(ticker_data_dict, index = [0])
        df = pd.concat([df, df_result_temp], ignore_index = True)
    
    #Calculating Industry Average
    print("Calculating Industry Averages")
    columns_in_question = ['EBITDA MARGIN (%)','EV/REVENUE (x)',
                           'EV/EBITDA (x)','PEG 5Y Expected (x)']
    
    industry_row_dict = {}
    for col in columns_in_question:
        if col == 'COMPANY NAME':
            industry_row_dict[col] = 'Average'
        elif col in columns_in_question:
            industry_row_dict[col] = df[col].mean()
        else:
            indsutry_row_dict[col] = None
    
    df_avg = pd.DataFrame(industry_row_dict, index = [0])
    df = pd.concat([df, df_avg], ignore_index = True)
    
    #Asssessing Intrinsic Value of company
    intrinsic_ratio = df.iloc[len(df) - 1 ]['EV/EBITDA (x)']
    print(intrinsic_ratio)
    df['Relative Fair Value'] = ((intrinsic_ratio * df['EBITDA ($M)'] * 1_000_000) - (df['TOTAL DEBT ($M)'] * 1_000_000)) / df['OUTSTANDING SHARES']
    
    #Include a column that suggests whether they are undervalued or overvalued
    
    #Completion of dataframes
    print("Results preview")
    print(df)
    
    #Export to Excel
    #df.to_excel(str(datetime.now()).replace(":", ".") + ".xlsx", sheet_name = "CCA")
    print("Success")
    time.sleep(5)
    
else:
    print("Incorrect Stock Ticker")



Input Company Ticker Symbol: sbux
['MCD', 'SBUX', 'CMG', 'YUM', 'QSR']
Getting data for:  MCD
MCD
Getting data for:  CMG
CMG
Getting data for:  YUM
YUM
Getting data for:  QSR
QSR
Getting data for:  SBUX
SBUX
Calculating Industry Averages
24.228
Results preview
  COMPANY NAME  SHARE PRICE ($/share) OUTSTANDING SHARES  MARKET CAP ($M)  \
0          MCD                285.530          730094016        208463.74   
1          CMG               2052.360           27589800         56624.21   
2          YUM                130.575          280087008         36572.36   
3          QSR                 71.825          311252000         32610.34   
4         SBUX                 99.470         1146400000        114032.41   
5          NaN                    NaN                NaN              NaN   

   TOTAL DEBT ($M)  TOTAL CASH ($M)  DILUTED EPS ($/share)  \
0          49999.7          3708.10                   8.33   
1           3771.6          1062.59                  32.04   
2          12

In [ ]:
#Alpha Vantage API KEY: IZPMUEDDXC0WJA3K

In [2]:
#Test 1
import requests

# Replace "YOUR_API_KEY" with your actual Polygon API key
api_key = "PSvIrcz1pUuI3P9KYA8qNvoenh89pc7U"
ticker = "AAPL" # Replace with the desired ticker symbol

url = f"https://api.polygon.io/v2/reference/financials/{ticker}/peers?apiKey={api_key}"
response = requests.get(url)
print(response.status_code)
if response.status_code == 200:
    data = response.json()
    peers = data["results"]
    print(f"Peers of {ticker}: {peers}")
else:
    print(f"Failed to retrieve peers for {ticker}. Error message: {response.text}")

404
Failed to retrieve peers for AAPL. Error message: 404 page not found


In [ ]:
#Test 2
from stocksymbol import StockSymbol

ap_key = '17396c59-63af-45dc-a690-9ea7c9779240'
ss = StockSymbol(api_key)


In [ ]:
#Test 3
#Selection of peer Universe (Top 5 companies with the highest market cap within the sector)
# Scrape the list of industries within the sector
base_url = "https://finance.yahoo.com"
industries_url = f"{base_url}/{sector_name}/{industry_name.lower()}"
response = requests.get(industries_url)

soup = BeautifulSoup(response.text, "html.parser")
industry_links = soup.find_all("a", class_="Fw(600)")
print(industry_links)

# Get the list of companies in each industry
companies = []
for link in industry_links:
    industry_url = base_url + link['href']
    response = requests.get(industry_url)
    soup = BeautifulSoup(response.text, "html.parser")
    table_rows = soup.find_all("tr", class_="simpTblRow")
    for row in table_rows:
        company_name = row.find("td", class_="Va(m)").text
        companies.append(company_name)

print(companies)

tech_market_cap_dict = {}
print(tickers.tickers)
for ticker in tickers.tickers:
    print(ticker)
    industry = ticker.info.get("industry", "")
    if industry_name in industry:
        tech_market_cap_dict[ticker.ticker] = ticker.info.get("marketCap", 0)

print(tech_market_cap_dict)
sorted_marketCap_dict = sorted(tech_market_cap_dict.items(), key = lambda x : x[1], reverse = True)
print(sorted_marketCap_dict)